# 06 Recommendation Systems
__Math 3280 - Data Mining__ : Snow College : Dr. Michael E. Olson

* Leskovec, Chapter 9
-----

In [3]:
import numpy as np
import pandas as pd

It will be helpful to think of the ratings as an n-dimensional vector where each component is a rating for a particular element (such as a specific actor).

## Utility Matrix

In [4]:
## Utility Matrix

movies = ['M01','M02','M03','M04','M05','M06','M07','M08','M09','M10','M11','M12','M13','M14','M15']

ratings = {
    'User A' : [0,1,1,1,0,1,1,0,1,0,0,1,1,1,1],
    'User B' : [1,1,0,0,0,0,1,1,0,1,0,1,0,0,1],
    'User C' : [0,0,0,1,1,0,0,0,0,1,0,0,0,0,1]
}

user_ratings = pd.DataFrame(ratings).transpose()
user_ratings.columns = movies

user_ratings

,M01,M02,M03,M04,M05,M06,M07,M08,M09,M10,M11,M12,M13,M14,M15
User A,0,1,1,1,0,1,1,0,1,0,0,1,1,1,1
User B,1,1,0,0,0,0,1,1,0,1,0,1,0,0,1
User C,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1


### Long Tail

## Content-Based Recommendations

### Item Profile

In [72]:
## Item Profile : Movie Database
actors_list = ['Julia Roberts','Robin Williams','Clint Eastwood','Ian McKellen','Movie Rating']

actors = {
    'M01' : [1,0,0,1,3],
    'M02' : [0,0,1,0,5],
    'M03' : [1,0,0,0,4],
    'M04' : [0,1,0,0,2],
    'M05' : [0,1,0,0,4],
    'M06' : [0,0,1,0,4],
    'M07' : [1,1,0,0,3],
    'M08' : [1,0,0,1,1],
    'M09' : [0,0,1,1,5],
    'M10' : [1,1,0,0,5],
    'M11' : [1,0,0,0,1],
    'M12' : [0,0,0,1,2],
    'M13' : [0,1,0,1,2],
    'M14' : [0,1,1,0,5],
    'M15' : [0,0,1,1,2]
}

movie_casts = pd.DataFrame(actors).transpose()
movie_casts.columns = actors_list

movie_casts

,Julia Roberts,Robin Williams,Clint Eastwood,Ian McKellen,Movie Rating
M01,1,0,0,1,3
M02,0,0,1,0,5
M03,1,0,0,0,4
M04,0,1,0,0,2
M05,0,1,0,0,4
M06,0,0,1,0,4
M07,1,1,0,0,3
M08,1,0,0,1,1
M09,0,0,1,1,5
M10,1,1,0,0,5


In [51]:
def cosine_distance(vector_1,vector_2):
    dotproduct = np.dot(vector_1, vector_2)
    norms = np.dot(vector_1, vector_1)*np.dot(vector_2, vector_2)
    return dotproduct/np.sqrt(norms)

In [52]:
## Example 9.2
alpha = 2
movie_1 = 'M07'
movie_2 = 'M10'

movie_comparison = movie_casts.loc[[movie_1, movie_2]]
movie_comparison['Movie Rating'] = movie_comparison['Movie Rating']*alpha
movie_comparison

,Julia Roberts,Robin Williams,Clint Eastwood,Ian McKellen,Movie Rating
M07,1,1,0,0,6
M10,1,1,0,0,10


In [53]:
cosine_distance(movie_comparison.loc[movie_1], movie_comparison.loc[movie_2])

0.995863477615015

In [54]:
alpha = 0.5
movie_comparison = movie_casts.copy()
movie_comparison['Movie Rating'] *= alpha

distances = pd.DataFrame(columns=movies)
for movie1 in movies:
    for movie2 in movies:
        distances.loc[movie1,movie2] = cosine_distance(movie_comparison.loc[movie1],movie_comparison.loc[movie2])

distances

,M01,M02,M03,M04,M05,M06,M07,M08,M09,M10,M11,M12,M13,M14,M15
M01,1.0,0.675566,0.867722,0.514496,0.650791,0.675566,0.764706,0.889297,0.802181,0.802181,0.759257,0.857493,0.70014,0.6333,0.792118
M02,0.675566,1.0,0.830455,0.656532,0.830455,1.0,0.675566,0.309492,0.937437,0.808135,0.415227,0.656532,0.536056,0.937437,0.909718
M03,0.867722,0.830455,1.0,0.632456,0.8,0.830455,0.867722,0.596285,0.778499,0.934199,0.8,0.632456,0.516398,0.778499,0.730297
M04,0.514496,0.656532,0.632456,1.0,0.948683,0.656532,0.857493,0.235702,0.615457,0.86164,0.316228,0.5,0.816497,0.86164,0.57735
M05,0.650791,0.830455,0.8,0.948683,1.0,0.830455,0.867722,0.298142,0.778499,0.934199,0.4,0.632456,0.774597,0.934199,0.730297
M06,0.675566,1.0,0.830455,0.656532,0.830455,1.0,0.675566,0.309492,0.937437,0.808135,0.415227,0.656532,0.536056,0.937437,0.909718
M07,0.764706,0.675566,0.867722,0.857493,0.867722,0.675566,1.0,0.565916,0.6333,0.971061,0.759257,0.514496,0.70014,0.802181,0.594089
M08,0.889297,0.309492,0.596285,0.235702,0.298142,0.309492,0.565916,1.0,0.522233,0.522233,0.745356,0.707107,0.57735,0.290129,0.544331
M09,0.802181,0.937437,0.778499,0.615457,0.778499,0.937437,0.6333,0.522233,1.0,0.757576,0.389249,0.86164,0.703526,0.878788,0.994937
M10,0.802181,0.808135,0.934199,0.86164,0.934199,0.808135,0.971061,0.522233,0.757576,1.0,0.700649,0.615457,0.703526,0.878788,0.710669


### User Profiles

#### User Profile with like/dislike ratings

In [16]:
## User Profile
## Example 9.3 : Movies given only likes or dislikes
dotproduct = np.dot(movie_casts['Julia Roberts'], user_ratings.loc['User A'])
norm = np.dot(user_ratings.loc['User A'], user_ratings.loc['User A'])
print(f"Dot Product of User Likes and Julia Roberts' Movies : {dotproduct}")
print(f"Norm of User Likes (number of movies liked)         : {norm}")
print(f"User weight to movies with Julia Roberts            : {dotproduct / norm}\n")


user_profile_likes = pd.DataFrame(columns=actors_list)

for actor_id in actors_list:
    for user_id in user_ratings.index:
        user_profile_likes.loc[user_id,actor_id] = np.dot(movie_casts[actor_id], user_ratings.loc[user_id]) / user_ratings.loc[user_id].sum()
        
user_profile_likes

Dot Product of User Likes and Julia Roberts' Movies : 2
Norm of User Likes (number of movies liked)         : 10
User weight to movies with Julia Roberts            : 0.2



,Julia Roberts,Robin Williams,Clint Eastwood,Ian McKellen,Movie Rating
User A,0.2,0.4,0.5,0.4,3.7
User B,0.571429,0.285714,0.285714,0.571429,3.285714
User C,0.25,0.75,0.25,0.25,3.75


#### User Profile with scaled ratings

In [114]:
## User Profile (Average rating)
### New Utility Matrix
ratings = {
    'User A' : [0,5,2,4,0,4,3,0,5,0,0,2,1,5,4],
    'User B' : [1,1,0,0,0,0,2,4,0,3,0,3,0,0,2],
    'User C' : [0,0,0,5,3,0,0,0,0,4,0,0,0,0,5]
    
    #'User A' : [np.nan,5,2,4,np.nan,4,3,np.nan,5,np.nan,np.nan,2,1,5,4],
    #'User B' : [1,1,np.nan,np.nan,np.nan,np.nan,2,4,np.nan,3,np.nan,3,np.nan,np.nan,2],
    #'User C' : [np.nan,np.nan,np.nan,5,3,np.nan,np.nan,np.nan,np.nan,4,np.nan,np.nan,np.nan,np.nan,5]
}

user_starratings = pd.DataFrame(ratings).transpose()
user_starratings.columns = movies

user_starratings

,M01,M02,M03,M04,M05,M06,M07,M08,M09,M10,M11,M12,M13,M14,M15
User A,0,5,2,4,0,4,3,0,5,0,0,2,1,5,4
User B,1,1,0,0,0,0,2,4,0,3,0,3,0,0,2
User C,0,0,0,5,3,0,0,0,0,4,0,0,0,0,5


With a 1-5 star rating, a 4 or 5 is really a positive rating, a 3 is average, and a 1 or 2 is a negative review. We want to normalize these so that the 1 and 2 are negative.

In [128]:
### Start by finding the average rating given by that user
user_avg_rating = user_starratings.replace(0,np.nan).loc['User A'].mean()
user_avg_rating

3.5

In [127]:
### Next find the movies with a given actor and the ratings the user has given it
actor_ratings_from_user = movie_casts['Julia Roberts'] * user_starratings.loc['User A']
actor_ratings_from_user

M01    0
M02    0
M03    2
M04    0
M05    0
M06    0
M07    3
M08    0
M09    0
M10    0
M11    0
M12    0
M13    0
M14    0
M15    0
dtype: int64

In [130]:
### Normalize the ratings by subtracting the average rating
actor_ratings_from_user = actor_ratings_from_user.apply(lambda x: 0 if x==0 else x-user_avg_rating)
actor_ratings_from_user

M01    0.0
M02    0.0
M03   -1.5
M04    0.0
M05    0.0
M06    0.0
M07   -0.5
M08    0.0
M09    0.0
M10    0.0
M11    0.0
M12    0.0
M13    0.0
M14    0.0
M15    0.0
dtype: float64

In [131]:
### The average is the score
actor_ratings_from_user.replace(0,np.nan).mean()

-1.0

In [135]:
a = 'Julia Roberts'
u = 'User A'

avg_rating = user_starratings.replace(0,np.nan).loc[u].mean()
(movie_casts[a] * user_starratings.loc[u]).apply(lambda x: 0 if x==0 else x-avg_rating).replace(0,np.nan).mean()

-1.0

In [137]:
### Apply for all users and actors
user_profile_ratings = pd.DataFrame(columns=actors_list)

for actor_id in actors_list:
    for user_id in user_ratings.index:
        avg_rating = user_starratings.replace(0,np.nan).loc[user_id].mean()  # Average rating given by user
        tmp = movie_casts[actor_id] * user_starratings.loc[user_id]          # Array of ratings given by user involving the given actor 
        user_profile_ratings.loc[user_id, actor_id] = tmp.apply(lambda x: 0 if x==0 else x-avg_rating).replace(0,np.nan).mean() # Subtract avg rating from ratings given, then take the mean
        

user_profile_ratings.drop('Movie Rating', axis=1, inplace=True)
user_profile_ratings

,Julia Roberts,Robin Williams,Clint Eastwood,Ian McKellen
User A,-1.0,-0.25,1.1,-0.5
User B,0.214286,0.214286,-0.785714,0.214286
User C,-0.25,-0.25,0.75,0.75


### Recommendations

#### Recommendations using binary (like/dislike) system

In [138]:
movie_casts.head()

,Julia Roberts,Robin Williams,Clint Eastwood,Ian McKellen,Movie Rating
M01,1,0,0,1,3
M02,0,0,1,0,5
M03,1,0,0,0,4
M04,0,1,0,0,2
M05,0,1,0,0,4


In [139]:
user_profile_likes.head()

,Julia Roberts,Robin Williams,Clint Eastwood,Ian McKellen,Movie Rating
User A,0.2,0.4,0.5,0.4,3.7
User B,0.571429,0.285714,0.285714,0.571429,3.285714
User C,0.25,0.75,0.25,0.25,3.75


In [148]:
cosine_distance(movie_casts.drop('Movie Rating', axis=1).loc['M06'],
                user_profile_likes.drop('Movie Rating', axis=1).loc['User A'])

0.6401843996644798

#### Recommendations using scaled (stars) system

In [149]:
user_profile_ratings.head()

,Julia Roberts,Robin Williams,Clint Eastwood,Ian McKellen
User A,-1.0,-0.25,1.1,-0.5
User B,0.214286,0.214286,-0.785714,0.214286
User C,-0.25,-0.25,0.75,0.75


In [150]:
cosine_distance(movie_casts.drop('Movie Rating', axis=1).loc['M06'],
                user_profile_ratings.loc['User A'])

0.6925914050226145

Advantages to Content-based approach
* No need for data from other users
* Able to cater to unique tastes
* Can include new and unpopular items in recommendations
* Can include explanations for recommendations (Because you liked 'A', you might like 'B')

Disadvantages to Content-based approach
* Difficult to find the appropriate features
  * Images, music, etc.
* Overspecialization (sticks to user's profile - doesn't go outside of that
* Can't take advantage of experience (quality judgments) from other users
* How do you make recommendations to a new user who doesn't have a profile?

-----
* Exercise 9.2.1 a-d
* Exercise 9.2.2 a
  * Calculate and interpret the cosine distances between each computer
* Exercise 9.2.3 a-b